# Text Generation with Neural Networks
In this notebook our objective is to demonstrate how to generate text using a character-based RNN working with a dataset of Shakespeare's  writing

# Notebook Overview
- Imports Dependencies
- Configurations
- Generating Predictions

## Imports Dependencies 

In [1]:
# Standard Library Imports
import logging
import warnings

# Third-Party Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GRU, InputLayer
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard


2025-04-11 11:40:57.961963: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 11:40:58.050895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744371658.094574    2019 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744371658.106452    2019 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-11 11:40:58.179583: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Configurations

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# Define global experiment and run names to be used throughout the notebook
MODEL_NAME = "tf_rnn_model.h5"

# Set up the paths
DATA_PATH = "../shakespeare.txt"
TENSORBOARD_PATH = "/phoenix/tensorboard/tensorlogs"


# Set up the chunk separator for text processing
CHUNK_SEPARATOR = "\n\n"

In [4]:
# === Create logger ===
logger = logging.getLogger("text-generation-TF-notebook")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                             datefmt="%Y-%m-%d %H:%M:%S") 

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [5]:
logger.info('Notebook execution started.')

2025-04-11 11:41:00 - INFO - Notebook execution started.


## Get Text Data

This is the text we'll use as a basis for our generations: let's try to generate 'Shakespearean' texts.

This text is from Shakespeare's Sonnet 1. It's one of the 154 sonnets written by William Shakespeare that were first published in 1609. This particular sonnet, like many others, discusses themes of beauty, procreation, and the transient nature of life, urging the beautiful to reproduce so their beauty can live on through their offspring.

In [6]:
path_to_file = DATA_PATH
text = open(path_to_file, 'r').read()

In [7]:
logger.info('First 600 chars: \n')
print(text[:600])

2025-04-11 11:41:00 - INFO - First 600 chars: 




                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else th


## Preparing textual data

We need to encode our data to give the model a proper numerical representation of our text.

In [8]:
# creates a set of unique characters found in the text
vocab = sorted(set(text))

### Text Vectorization

In [9]:
char_to_int = {u:i for i, u in enumerate(vocab)}
# assigns a unique integer to each character in a dictionary format, 
# creating a mapping that can later be used to transform encoded predictions back into characters

In [10]:
int_to_char = np.array(vocab)
# reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers, which is used to encode the text.

In [11]:
encoded_text = np.array([char_to_int[c] for c in text])
# encodes the entire text as an array of integers, with each integer representing the character at that position
# in the text according to the encoder dictionary

## Creating Training Batches

Training batches are a way of dividing the dataset into smaller, manageable groups of data points that are fed into a machine learning model during the training process.

In [12]:
seq_len = 120 # length of sequence for a training example
total_num_seq = len(text)//(seq_len+1) # total number of training examples

# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

I0000 00:00:1744371661.633890    2019 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2268 MB memory:  -> device: 0, name: NVIDIA T600, pci bus id: 0000:01:00.0, compute capability: 7.5


In [13]:
def create_seq_targets(seq):
    """
    Function that takes a sequence as input, duplicates, and shifts it to align the input and label. 

    Args:
        seq: sequence of characters

    Returns:
        The text input and corresponding target.
    """
    try:
        input_txt = seq[:-1]
        target_txt = seq[1:]
        return input_txt, target_txt
    except Exception as e:
            logger.error(f"Error creating sequences of targets: {str(e)}")

In [14]:
dataset = sequences.map(create_seq_targets)

In [15]:
# Batch size
batch_size = 128
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

## Creating the GRU Model

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embed_dim = 64
# Number of RNN units
rnn_neurons = 1026

In [17]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [18]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    """Architecture to create the model.

    Args:
        vocab_size: Length of the vocabulary in chars.
        embed_dim: Embedding dimension.
        rnn_neurons: Number of RNN units.
        batch_size: Size of the batchs.

    Returns:
        Model.
    """
    try:
        model = Sequential()
        model.add(InputLayer(batch_shape=(batch_size, None)))
        
        model.add(Embedding(input_dim=vocab_size, output_dim=embed_dim))

        model.add(GRU(rnn_neurons,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'))

        model.add(Dense(vocab_size))
        model.compile(optimizer='adam', loss=sparse_cat_loss)
        logger.info("Model architecture created successfully")
        return model
    except Exception as e:
            logger.error(f"Error creating model architecture: {str(e)}")

model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size)
model.summary()


2025-04-11 11:41:02 - INFO - Model architecture created successfully


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (128, None, 64)        │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (128, None, 1026)      │     3,361,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (128, None, 84)        │        86,268 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,452,820 (13.17 MB)

 Trainable params: 3,452,820 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

## Instance of the Model

In [19]:
model = create_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    rnn_neurons=rnn_neurons,
    batch_size=batch_size
)


2025-04-11 11:41:02 - INFO - Model architecture created successfully


In [20]:
# TensorBoard
log_dir = TENSORBOARD_PATH
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

## Training the model

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


I0000 00:00:1744371663.922157    2115 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-11 11:41:04.097332: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
epochs = 20
model.fit(dataset,epochs=epochs, callbacks=[tensorboard_callback])

Epoch 1/2
351/351 ━━━━━━━━━━━━━━━━━━━━ 102s 286ms/step - loss: 2.9884
Epoch 2/2
351/351 ━━━━━━━━━━━━━━━━━━━━ 104s 294ms/step - loss: 1.6718


## Saving the Model

In [24]:
model_name = MODEL_NAME

In [25]:
model.save(f'models/{model_name}') 
logger.info("Model saved")

2025-04-11 11:44:30 - INFO - Model saved


## Load Model

In [27]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights(f'models/{model_name}')

model.build(tf.TensorShape([1, None]))

2025-04-11 11:44:45 - INFO - Model architecture created successfully


# Generating Predictions

In [28]:
def generate_text(model, start_seed="The ", gen_size=100, temp=1.0):
    """
    Generates a sequence of text using the trained character-level language model.

    Args:
        model: Model created on function create_model
        start_seed: Set of characters that will be the beginning of the text. 
        gen_size : Number of characters. Defaults to 100.
        temp: Controls the randomness of the predictions made by the model.

    Returns:
        The full generated text including the seed and the newly predicted characters.
    """
    try:
        num_generate = gen_size
        input_eval = [char_to_int[s] for s in start_seed]
        input_eval = tf.expand_dims(input_eval, 0)
        text_generated = []
        temperature = temp


        for i in range(num_generate):
            predictions = model(input_eval)
            predictions = tf.squeeze(predictions, 0)
            predictions = predictions / temperature
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            input_eval = tf.expand_dims([predicted_id], 0)
            text_generated.append(int_to_char[predicted_id])

        return start_seed + ''.join(text_generated)
    except Exception as e:
            logger.error(f"Error making predictions: {str(e)}")


#### Generating a text with 1000 chars starting with word 'Confidence'

In [29]:
for layer in model.layers:
    if hasattr(layer, 'reset_states'):
        layer.reset_states()
print(generate_text(model, start_seed="Confidence ", gen_size=1000))

Confidence do our guns stould.
    Alcect with a vain.
  KING RINTHER: Even uneep of the time of the
    sades thy block-body  four own silve I love allows him. Am, thy worsh goush so excy the bass?
  PROSPERO. So, I shude eye.
  PAUSNE. Why, he thou?
  LURICTIS. So, not! Make King's more acammed!
    Pivs he had'ling on of goferation; yet your pricketiver
    to eats?  
  DEMETIUS. That's not the can, ratures, my before thee as this trount again,
                    stand them take home. But, beat it.
  ASTIDON. I shall me till be rove!
     Within, gov'dlen go, when fur lick my defit a meant and my father.
    Captark you call'st in other sing.
  PRILSUS. Ay, and elemy't were such anot;  
    I do not live a mance book; and so or faiting for ware, for death, let it saf she baid?
  NGR. Heris the depuist of that letters upon the
    to see delsears; a'l I liad up Slawors meam;
    Businels? Hilling lord what this liktled hyord up to
    ant your bugoes; such we hopes the King draw I w

#### Generating a text with 1000 chars starting with word 'Love'

In [30]:
print(generate_text(model, start_seed="Love ", gen_size=1000))

Love is a  as a falcies armar
MESMERCIANED BY WORLD LIBRARY, JAONCELLOWER, good turm:
  OMERIO. Mehay, and neie borted, Sir, a speaks raughmy not voice of empant

                                  Exit
  GUKES. And now, marry? Not curcimes not
  Whibled betweent a tomm am any
    beggaarly, you to you. Then from noth of gownref is
    to this thinces. Then thy that, the lad-stiols levive of a king again?
  LUCETIZAN. Let it gower. , such will men!  
  TIMON. And like my conseuged Roveranat
    Than brother's waur resume, the godswarding. An parse of Apail.

                          Enter COMIS (UTRIBUTED SO Lequs your loyalts,
    for this told makes for entreat wewerd breact from mine.
    Then I ef glasply too mont To your Parron is he bay'st do me love gheat grian the wing are,
    Atten will see not? Sterfia woot?
  MASTOR. My geterultany. If what I am so our Those there to Burg with

    Efes this his bloods the marretibinate; whice moneyom my false.
    I am not was abouted but 

Built with ❤️ using Z by HP AI Studio.